In [1]:
# 重启
# !kill -9 -1
# 安装包
!pip install surprise

     |████████████████████████████████| 6.5MB 2.5MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675364 sha256=f2918c12577c01432bff49c7434ed46f75f3fb1ca79808c7998630ff997114f6
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [2]:
from google.colab import drive
drive.mount('/content/drive')
# 创建数据文件夹
!mkdir /content/dataset
# 复制文件到本地
!cp /content/drive/My\ Drive/Dataset/netflix-prize-data.zip /content/dataset/netflix-prize-data.zip
# 切换当前目录
%cd /content/dataset/
# 解压缩文件
!unzip netflix-prize-data.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/dataset
Archive:  netflix-prize-data.zip
  inflating: README                  
  inflating: combined_data_1.txt     
  inflating: combined_data_2.txt     
  inflating: combined_data_3.txt     
  inflating: combined_data_4.txt     
  inflating: movie_titles.csv        
  inflating: probe.txt               
  inflating: qualifying.txt          


In [0]:
# 导入需要用到的包
import pandas as pd
import numpy as np
import time

from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, SlopeOne, SVD, SVDpp
from surprise import accuracy
from surprise.model_selection import KFold

In [4]:
# 回收内存 
import gc
gc.collect()
# 查看内存中的变量
dir()
# %cd /content/dataset/

['BaselineOnly',
 'Dataset',
 'In',
 'KFold',
 'Out',
 'Reader',
 'SVD',
 'SVDpp',
 'SlopeOne',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_exit_code',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 '_sh',
 'accuracy',
 'drive',
 'exit',
 'gc',
 'get_ipython',
 'np',
 'pd',
 'quit',
 'time']

## 数据预处理

In [0]:
# 处理训练集的函数
def data_cleaning(data):
  data['movie_id']=data[0].map(lambda x:int(x[:-1]) if ':' in x else np.NaN)
  data=data.fillna(method='ffill')
  data=data[data[0].map(lambda x: ':' not in x)]
  data['user_id']=data[0].map(lambda x:x.split(',')[0]).astype(np.int)
  data['rating']=data[0].map(lambda x:x.split(',')[1]).astype(np.float)
  data=data.drop([data.columns[0]],axis=1)
  #data=data.set_index('user_id')

  return data
# 处理probe数据集的函数
def probe_cleaning(data):
  data['movie_id']=data[0].map(lambda x:int(x[:-1]) if ':' in x else np.NaN)
  data=data.fillna(method='ffill')
  data=data[data[0].map(lambda x: ':' not in x)]
  data.columns=['user_id','movie_id']
  data['user_id']=data['user_id'].astype(np.int)
  data=data[['movie_id','user_id']]

  return data

In [0]:
# 提取所有训练
data1=pd.read_csv('./combined_data_1.txt',sep=' ',header=None)
data2=pd.read_csv('./combined_data_2.txt',sep=' ',header=None)
data3=pd.read_csv('./combined_data_3.txt',sep=' ',header=None)
data4=pd.read_csv('./combined_data_4.txt',sep=' ',header=None)
data1 = data_cleaning(data1)
data2 = data_cleaning(data2)
data3 = data_cleaning(data3)
data4 = data_cleaning(data4)

In [7]:
# 将训练数据合并
data_all=pd.concat([data1,data2,data3,data4],ignore_index=True)
del data1, data2, data3, data4
data_all

,movie_id,user_id,rating
0,1.0,1488844,3.0
1,1.0,822109,5.0
2,1.0,885013,4.0
3,1.0,30878,4.0
4,1.0,823519,3.0
...,...,...,...
100480502,17770.0,1790158,4.0
100480503,17770.0,1608708,3.0
100480504,17770.0,234275,1.0
100480505,17770.0,255278,4.0


In [8]:
# 提取probe数据 作为测试集
probe=probe_cleaning(pd.read_csv('./probe.txt',sep=' ',header=None))
# probe数据集补全得分
probe=pd.merge(probe,data_all)
probe

,movie_id,user_id,rating
0,1.0,30878,4.0
1,1.0,2647871,4.0
2,1.0,1283744,3.0
3,1.0,2488120,5.0
4,1.0,317050,5.0
...,...,...,...
1408390,9997.0,2328701,3.0
1408391,9998.0,1288730,3.0
1408392,9998.0,2536567,4.0
1408393,9998.0,1107317,5.0


In [9]:
# 将probe数据从训练集中删除作为测试集
data_all=pd.concat([data_all,probe],ignore_index=True)
data_all=data_all.drop_duplicates(subset=['user_id','movie_id'],keep=False)
data_all

,movie_id,user_id,rating
0,1.0,1488844,3.0
1,1.0,822109,5.0
2,1.0,885013,4.0
4,1.0,823519,3.0
5,1.0,893988,3.0
...,...,...,...
100480502,17770.0,1790158,4.0
100480503,17770.0,1608708,3.0
100480504,17770.0,234275,1.0
100480505,17770.0,255278,4.0


In [10]:
# 压缩数据 m=4 和 n=4 可以压缩很多数据 (2,2)本地需要600M，试下直接fit模型 (4.4)可以使用kflod交叉验证
def data_sample(data,m,n):
  data=data[data['user_id']%n==0]
  data=data[data['movie_id']%m==0]
  return data
data_all = data_sample(data_all,2,2)
probe = data_sample(probe,2,2)
data_all
probe

,movie_id,user_id,rating
26,1000.0,977808,3.0
27,1000.0,1010534,2.0
31,1000.0,1960212,2.0
32,1000.0,97460,5.0
33,1000.0,2623506,4.0
...,...,...,...
1408381,9996.0,570792,4.0
1408382,9996.0,1619158,5.0
1408383,9996.0,2571420,1.0
1408385,9996.0,1206224,5.0


In [13]:
data_all

,movie_id,user_id,rating
2705,4.0,1544320,1.0
2707,4.0,732556,3.0
2710,4.0,303948,2.0
2711,4.0,2418760,4.0
2714,4.0,1519860,3.0
...,...,...,...
100472819,17768.0,558252,2.0
100472820,17768.0,780984,2.0
100472829,17768.0,295196,2.0
100472831,17768.0,1883788,3.0


In [0]:
# 数据保存到CSV
data_all.to_csv("netflix_train_2_2.csv", index=False)
probe.to_csv("netflix_prob_2_2.csv", index=False)

In [0]:
data_all = data_sample(data_all,4,4)
probe = data_sample(probe,4,4)
# 数据保存到CSV
data_all.to_csv("netflix_train_4_4.csv", index=False)
probe.to_csv("netflix_prob_4_4.csv", index=False)
del data_all, probe

## 训练模型

In [0]:
# 从csv导入数据，为了降低内存使用
reader = Reader(line_format='item user rating', sep=',', skip_lines=1)
# load_from_file 直接从csv中导入数据
data = Dataset.load_from_file('netflix_train_4_4.csv',reader=reader)

# 讲所有训练就作为训练集后面不用Kfold
# train_set = data.build_full_trainset()

In [0]:
# 从DF导入数据
reader = Reader(line_format='item user rating', sep=',', skip_lines=1)
# load_from_df  直接从DataFrame 调用数据
data = Dataset.load_from_df(data_all[['movie_id', 'user_id', 'rating']], reader=reader)
del data_all
# 讲所有训练就作为训练集后面不用Kfold
# train_set = data.build_full_trainset()

In [0]:
# Kfold交叉验证
def kftrain(algo, data, k):
  kf = KFold(n_splits=k)
  for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)
  return algo

# 测试集进行测试
def probe_test(algo):
  probe = pd.read_csv("netflix_prob_4_4.csv")
  # 在porbe数据集预测并评分
  probe_test=[]
  for i in probe.values:
      tup=(i[0],i[1],i[2])
      probe_test.append(tup)
  # 在测试集预测
  probe_predictions = algo.test(probe_test)
  print("使用porbe测试")
  # 计算RMSE
  accuracy.rmse(probe_predictions, verbose=True)
  return True

In [15]:
# ALS优化 BaselineOnly 算法
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
Base_algo = BaselineOnly(bsl_options=bsl_options)

# 开始时间
time1=time.time()

# 直接使用数据集作为训练集
trainset = data.build_full_trainset()
Base_algo.fit()

# Kfold交叉验证
# Base_algo = kftrain(Base_algo, data, 3)

# 使用测试集测试
probe_test(Base_algo)

# 结束时间并且打印耗时
time2=time.time()
print("消耗时间：",time2-time1)

Estimating biases using als...
RMSE: 0.9277
Estimating biases using als...
RMSE: 0.9272
Estimating biases using als...
RMSE: 0.9280
使用porbe测试
RMSE: 1.1231
消耗时间： 213.42427897453308


In [16]:
# 使用funkSVD
fSVDalgo = SVD(biased=False)

# 开始时间
time1=time.time()

# 直接使用数据集作为训练集
# trainset = data.build_full_trainset()
# fSVDalgo.fit()

# Kfold交叉验证
fSVDalgo = kftrain(fSVDalgo, data, 3)

# 使用测试集测试
probe_test(fSVDalgo)

# 结束时间并且打印耗时
time2=time.time()
print("消耗时间：",time2-time1)

RMSE: 1.0153
RMSE: 1.0158
RMSE: 1.0152
使用porbe测试
RMSE: 1.1231
消耗时间： 1014.4577970504761


In [17]:
# 使用biasSVD
bSVDalgo = SVD()

# 开始时间
time1=time.time()

# 直接使用数据集作为训练集
# trainset = data.build_full_trainset()
# bSVDalgo.fit()

# Kfold交叉验证
bSVDalgo = kftrain(bSVDalgo, data, 3)

# 使用测试集测试
probe_test(bSVDalgo)

# 结束时间并且打印耗时
time2=time.time()
print("消耗时间：",time2-time1)

RMSE: 0.8984
RMSE: 0.8967
RMSE: 0.8981
使用porbe测试
RMSE: 1.1232
消耗时间： 1057.7897398471832


In [0]:
# 使用SVD++
SVDppalgo = SVDpp()

# 开始时间
time1=time.time()

# 直接使用数据集作为训练集
# trainset = data.build_full_trainset()
# algo.fit()

# Kfold交叉验证
SVDppalgo = kftrain(SVDppalgo, data, 3)

# 使用测试集测试
probe_test(SVDppalgo)

# 结束时间并且打印耗时
time2=time.time()
print("消耗时间：",time2-time1)